# Live Daten einlesen

In [89]:
import pandas as pd
import json

json_file_path = "./data/msr.json"

# read JSON
with open(json_file_path, 'r') as file:
    data = json.load(file)

# create an empty list
df_list = []

# get out all sensorMeasurements data and adds it to the rest of the detector measurements
for entry in data['detector_measurements']:
    measurements_data = entry['sensorMeasurements']
    entry.pop('sensorMeasurements')  # delete 'sensorMeasurements'(we only need each measurement)
    for measurement in measurements_data:
        entry.update(measurement)  # add details of sensorMeasurements entry
        df_list.append(entry.copy())  # append copy of entry to list

# create DataFrame
df = pd.DataFrame(df_list)

# show DataFrame
df

,id,time,value,hasError,errorReason,index,kind,numberOfInputValuesUsed
0,CH:0542.05,2023-12-02T12:37:00.000000Z,0.000000,True,VD_OFFLINE,0,,NaN
1,CH:0026.03,2023-12-02T12:37:00.000000Z,1320.000000,False,,11,trafficFlow,NaN
2,CH:0026.03,2023-12-02T12:37:00.000000Z,96.699997,False,,12,trafficSpeed,22.0
3,CH:0026.03,2023-12-02T12:37:00.000000Z,0.000000,False,,21,trafficFlow,22.0
4,CH:0026.03,2023-12-02T12:37:00.000000Z,0.000000,False,,1,trafficFlow,22.0
...,...,...,...,...,...,...,...,...
6342,CH:0505.03,2023-12-02T12:37:00.000000Z,0.000000,False,,1,trafficFlow,22.0
6343,CH:0505.02,2023-12-02T12:37:00.000000Z,900.000000,False,,11,trafficFlow,NaN
6344,CH:0505.02,2023-12-02T12:37:00.000000Z,115.300003,False,,12,trafficSpeed,15.0
6345,CH:0505.02,2023-12-02T12:37:00.000000Z,0.000000,False,,21,trafficFlow,15.0


In [90]:
# make sure that time is a datetime
df['time'] = pd.to_datetime(df['time'])
df

,id,time,value,hasError,errorReason,index,kind,numberOfInputValuesUsed
0,CH:0542.05,2023-12-02 12:37:00+00:00,0.000000,True,VD_OFFLINE,0,,NaN
1,CH:0026.03,2023-12-02 12:37:00+00:00,1320.000000,False,,11,trafficFlow,NaN
2,CH:0026.03,2023-12-02 12:37:00+00:00,96.699997,False,,12,trafficSpeed,22.0
3,CH:0026.03,2023-12-02 12:37:00+00:00,0.000000,False,,21,trafficFlow,22.0
4,CH:0026.03,2023-12-02 12:37:00+00:00,0.000000,False,,1,trafficFlow,22.0
...,...,...,...,...,...,...,...,...
6342,CH:0505.03,2023-12-02 12:37:00+00:00,0.000000,False,,1,trafficFlow,22.0
6343,CH:0505.02,2023-12-02 12:37:00+00:00,900.000000,False,,11,trafficFlow,NaN
6344,CH:0505.02,2023-12-02 12:37:00+00:00,115.300003,False,,12,trafficSpeed,15.0
6345,CH:0505.02,2023-12-02 12:37:00+00:00,0.000000,False,,21,trafficFlow,15.0


In [91]:
# index mapping to vehicle type

#sk: Vehicle Type würde ich nicht in die Live Daten integrieren, sondern eine im Filter hinterlegen: wenn "Lorry" ausgewählt ist, dann nimm alle Zeilen mit index 21 und index 22.. etc.

In [94]:
# add canton

# get measurement_station id
df['id_measurement_station'] = df['id'].str.split('.').str[0]

# read mst data
json_file_path = "./data/mst.json"
with open(json_file_path, 'r') as file:
    json_data = json.load(file)

# Create mapping
canton_mapping = {entry['id']: entry['canton'] for entry in json_data}
eastLv95_mapping = {entry['id']: entry['eastLv95'] for entry in json_data}
northLv95_mapping = {entry['id']: entry['northLv95'] for entry in json_data}

# Map 'canton' values to the DataFrame
df['canton'] = df['id_measurement_station'].map(canton_mapping)

df['eastLv95'] = df['id_measurement_station'].map(eastLv95_mapping)
df['eastLv95'] = df['eastLv95'].astype(float)

df['northLv95'] = df['id_measurement_station'].map(northLv95_mapping )
df['northLv95'] = df['northLv95'].astype(float)

# Display the updated DataFrame
df

,id,time,value,hasError,errorReason,index,kind,numberOfInputValuesUsed,id_measurement_station,canton,eastLv95,northLv95,cluster_kmeans,cluster
0,CH:0542.05,2023-12-02 12:37:00+00:00,0.000000,True,VD_OFFLINE,0,,NaN,CH:0542,AG,2668970.0,1254246.0,0,-1
1,CH:0026.03,2023-12-02 12:37:00+00:00,1320.000000,False,,11,trafficFlow,NaN,CH:0026,LU,2666381.0,1205794.0,2,-1
2,CH:0026.03,2023-12-02 12:37:00+00:00,96.699997,False,,12,trafficSpeed,22.0,CH:0026,LU,2666381.0,1205794.0,2,-1
3,CH:0026.03,2023-12-02 12:37:00+00:00,0.000000,False,,21,trafficFlow,22.0,CH:0026,LU,2666381.0,1205794.0,2,-1
4,CH:0026.03,2023-12-02 12:37:00+00:00,0.000000,False,,1,trafficFlow,22.0,CH:0026,LU,2666381.0,1205794.0,2,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6342,CH:0505.03,2023-12-02 12:37:00+00:00,0.000000,False,,1,trafficFlow,22.0,CH:0505,VD,2507348.0,1141026.0,9,-1
6343,CH:0505.02,2023-12-02 12:37:00+00:00,900.000000,False,,11,trafficFlow,NaN,CH:0505,VD,2507348.0,1141026.0,9,-1
6344,CH:0505.02,2023-12-02 12:37:00+00:00,115.300003,False,,12,trafficSpeed,15.0,CH:0505,VD,2507348.0,1141026.0,9,-1
6345,CH:0505.02,2023-12-02 12:37:00+00:00,0.000000,False,,21,trafficFlow,15.0,CH:0505,VD,2507348.0,1141026.0,9,-1


In [95]:
# Clustering region (not cantone)
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

#k-means
desired_number_of_clusters = 10
kmeans = KMeans(n_clusters=desired_number_of_clusters)
df['cluster_kmeans'] = kmeans.fit_predict(df[['eastLv95', 'northLv95']])

# DBSCAN (Density-Based Spatial Clustering of Applications with Noise):
desired_epsilon = 0.1
desired_min_samples = 30
dbscan = DBSCAN(eps=desired_epsilon, min_samples=desired_min_samples)
df['cluster'] = dbscan.fit_predict(df[['eastLv95', 'northLv95']])

C:\Users\sarak\AppData\Local\Programs\Python\fhgr-cp2-astra-dashboard\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [96]:
df

#sk: müsste auf einer Karte angeschaut werden

,id,time,value,hasError,errorReason,index,kind,numberOfInputValuesUsed,id_measurement_station,canton,eastLv95,northLv95,cluster_kmeans,cluster
0,CH:0542.05,2023-12-02 12:37:00+00:00,0.000000,True,VD_OFFLINE,0,,NaN,CH:0542,AG,2668970.0,1254246.0,0,-1
1,CH:0026.03,2023-12-02 12:37:00+00:00,1320.000000,False,,11,trafficFlow,NaN,CH:0026,LU,2666381.0,1205794.0,5,-1
2,CH:0026.03,2023-12-02 12:37:00+00:00,96.699997,False,,12,trafficSpeed,22.0,CH:0026,LU,2666381.0,1205794.0,5,-1
3,CH:0026.03,2023-12-02 12:37:00+00:00,0.000000,False,,21,trafficFlow,22.0,CH:0026,LU,2666381.0,1205794.0,5,-1
4,CH:0026.03,2023-12-02 12:37:00+00:00,0.000000,False,,1,trafficFlow,22.0,CH:0026,LU,2666381.0,1205794.0,5,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6342,CH:0505.03,2023-12-02 12:37:00+00:00,0.000000,False,,1,trafficFlow,22.0,CH:0505,VD,2507348.0,1141026.0,1,-1
6343,CH:0505.02,2023-12-02 12:37:00+00:00,900.000000,False,,11,trafficFlow,NaN,CH:0505,VD,2507348.0,1141026.0,1,-1
6344,CH:0505.02,2023-12-02 12:37:00+00:00,115.300003,False,,12,trafficSpeed,15.0,CH:0505,VD,2507348.0,1141026.0,1,-1
6345,CH:0505.02,2023-12-02 12:37:00+00:00,0.000000,False,,21,trafficFlow,15.0,CH:0505,VD,2507348.0,1141026.0,1,-1
